# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle
import random
from random import sample

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import Conditional_VAE as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/train_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(777)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
#dev_dir = INPUT_ROOT + "/dev_data"
#add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = ['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/CenterLossVAE/efficientnet/ex21/config.yaml'

In [7]:
torch.cuda.is_available()

True

In [8]:
ext_data = pd.read_pickle(f'{INPUT_ROOT}/valve_features.pkl')

In [9]:
#train_features = ext_data['train']['features']

In [10]:
#train_features.shape

## training

In [11]:
def random_feature_selection(x, t_d=3776, d=2048):
    idx = torch.tensor(sample(range(0, t_d), d)) # t_d(特徴量)の中からランダムにd個サンプリングする(ぜんぶやると重いから) # わからん
    # 0~t_d idxの中からランダムにd個サンプリングする
    x = np.take(x, idx, 1)
    return x

In [12]:
#ext_data['train'].keys()

In [13]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    #for machine in machine_types:
    #    if machine_type == machine:
    #        ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine}_features.pkl')
    #    else:
    #        ext_data_ = pd.read_pickle(f'{INPUT_ROOT}/{machine}_features.pkl')
    #        ext_data['train']['features'] = np.concatenate([ext_data['train']['features'],
    #                                                        ext_data_['train']['features']], axis=0)
    #        ext_data['train']['labels'] = np.concatenate([ext_data['train']['labels'],
    #                                                     ext_data_['train']['labels']], axis=0)
    #        ext_data['train']['wav_names'] = np.concatenate([ext_data['train']['wav_names'],
    #                                                         ext_data_['train']['wav_names']], axis=0)
            
    #t_d = 3776    # 全特徴量数
    #d = 1024      # 使う特徴量数
    #for phase in ['train', 'valid_source', 'valid_target']:
    #    x = ext_data[phase]['features']
    #    x = random_feature_selection(x)
    #    ext_data[phase]['features'] = x
    #    x = ext_data[phase]['features'][:,512:2048]
    #    ext_data[phase]['features'] = x
    dataloaders_dict = modeler.make_dataloader(ext_data)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['train']['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    num_classes = 6
    net = Model(in_features, mid_size, latent_size, num_classes)
    optimizer = optim.Adam(net.parameters(), lr=1e-4)
    num_epochs = config['param']['num_epochs']
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
    #                                          max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, num_epochs, writer, model_out_path, score_out_path, pred_out_path)
    
    com.toc()

In [14]:
machine_types

['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']

In [15]:
for machine_type in machine_types:
    run(machine_type)

2021-06-01 20:35:43,844 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyCar
2021-06-01 20:35:43,844 - 00_train.py - INFO - MAKE DATA_LOADER
2021-06-01 20:35:44,771 - 00_train.py - INFO - TRAINING
2021-06-01 20:35:48,515 - pytorch_modeler.py - INFO - train
  0%|          | 0/6 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 6/6 [00:00<00:00,  9.32it/s]
2021-06-01 20:35:49,162 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 1/1 [00:00<00:00, 54.93it/s]
2021-06-01 20:35:49,203 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 1/1 [00:00<00:00, 52.35it/s]


,AUC,pAUC
Source_0,0.605700,0.557368
Source_1,0.529800,0.514211
Source_2,0.544700,0.505263
Target_0,0.631900,0.582632
Target_1,0.537900,0.490526
Target_2,0.523500,0.504211
mean,0.562250,0.525702
h_mean,0.559378,0.523729


2021-06-01 20:35:51,974 - pytorch_modeler.py - INFO - epoch:1/300, train_losses:2587.614787, val_AUC_hmean:0.559378, val_pAUC_hmean:0.523729, best_flag:True
2021-06-01 20:35:51,975 - pytorch_modeler.py - INFO - train
100%|██████████| 6/6 [00:00<00:00, 23.04it/s]
2021-06-01 20:35:52,237 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 1/1 [00:00<00:00, 57.45it/s]
2021-06-01 20:35:52,274 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 1/1 [00:00<00:00, 58.33it/s]


...

2021-06-01 20:55:14,028 - pytorch_modeler.py - INFO - train
100%|██████████| 6/6 [00:00<00:00, 19.83it/s]
2021-06-01 20:55:14,332 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 1/1 [00:00<00:00, 63.19it/s]
2021-06-01 20:55:14,376 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 1/1 [00:00<00:00, 45.60it/s]
2021-06-01 20:55:14,427 - pytorch_modeler.py - INFO - epoch:260/300, train_losses:335.531560, val_AUC_hmean:0.694077, val_pAUC_hmean:0.583780, best_flag:False
2021-06-01 20:55:14,428 - pytorch_modeler.py - INFO - train
100%|██████████| 6/6 [00:00<00:00, 20.24it/s]
2021-06-01 20:55:14,726 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 1/1 [00:00<00:00, 51.48it/s]
2021-06-01 20:55:14,770 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 1/1 [00:00<00:00, 39.22it/s]
2021-06-01 20:55:14,827 - pytorch_modeler.py - INFO - epoch:261/300, train_losses:334.342219, val_AUC_hmean:0.700378, val_pAUC_hmean:0.589918, best_flag:False
2021-06-01 20:55:1

2021-06-01 20:55:21,030 - pytorch_modeler.py - INFO - train
100%|██████████| 6/6 [00:00<00:00, 15.18it/s]
2021-06-01 20:55:21,427 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 1/1 [00:00<00:00, 59.13it/s]
2021-06-01 20:55:21,470 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 1/1 [00:00<00:00, 46.82it/s]
2021-06-01 20:55:21,523 - pytorch_modeler.py - INFO - epoch:277/300, train_losses:311.999395, val_AUC_hmean:0.700153, val_pAUC_hmean:0.592186, best_flag:False
2021-06-01 20:55:21,524 - pytorch_modeler.py - INFO - train
100%|██████████| 6/6 [00:00<00:00, 19.87it/s]
2021-06-01 20:55:21,830 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 1/1 [00:00<00:00, 47.03it/s]
2021-06-01 20:55:21,877 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 1/1 [00:00<00:00, 44.65it/s]
2021-06-01 20:55:21,929 - pytorch_modeler.py - INFO - epoch:278/300, train_losses:311.656977, val_AUC_hmean:0.692894, val_pAUC_hmean:0.585062, best_flag:False
2021-06-01 20:55:2

2021-06-01 20:55:28,163 - pytorch_modeler.py - INFO - train
100%|██████████| 6/6 [00:00<00:00, 19.85it/s]
2021-06-01 20:55:28,468 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 1/1 [00:00<00:00, 59.04it/s]
2021-06-01 20:55:28,512 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 1/1 [00:00<00:00, 50.85it/s]
2021-06-01 20:55:28,560 - pytorch_modeler.py - INFO - epoch:294/300, train_losses:291.214177, val_AUC_hmean:0.700761, val_pAUC_hmean:0.586741, best_flag:False
2021-06-01 20:55:28,561 - pytorch_modeler.py - INFO - train
100%|██████████| 6/6 [00:00<00:00, 24.43it/s]
2021-06-01 20:55:28,808 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 1/1 [00:00<00:00, 59.40it/s]
2021-06-01 20:55:28,847 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 1/1 [00:00<00:00, 50.71it/s]
2021-06-01 20:55:28,898 - pytorch_modeler.py - INFO - epoch:295/300, train_losses:290.383769, val_AUC_hmean:0.698843, val_pAUC_hmean:0.586133, best_flag:False
2021-06-01 20:55:2

,AUC,pAUC
Source_0,0.774400,0.623684
Source_1,0.642700,0.532632
Source_2,0.856800,0.699474
Target_0,0.784000,0.668947
Target_1,0.660000,0.598421
Target_2,0.639300,0.511579
mean,0.726200,0.605789
h_mean,0.716909,0.598135


elapsed time: 164.153806210 [sec]


In [16]:
#run(machine_types[0])